# [Advent of Code 2020 Day 10](https://adventofcode.com/2020/day/10)

This looks nasty.

## Initial setup

In [ ]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Main Solver

In [ ]:
def solve(prob, filename):
    lines = []
    gen = yield_line(filename)

    for line in gen:
        lines.append(line)

    if prob == 1:
        return 0x3f3f3f3f + 1
    elif prob == 2:
        return 0x3f3f3f3f + 2
    else:
        print("Invalid problem code")
        exit()

In [ ]:
%%ipytest
def test_solve():
    assert solve(1, "example") == 0x3f3f3f3f + 1
    assert solve(1, "input") == 0x3f3f3f3f + 1
    assert solve(2, "example") == 0x3f3f3f3f + 2
    assert solve(2, "input") == 0x3f3f3f3f + 2